In [36]:
from abc import ABC, abstractmethod
import re

import pandas as pd
import os
from sqlalchemy import create_engine, MetaData
from sqlalchemy.orm import sessionmaker
from dotenv import dotenv_values

## Settings database and create SQLALCHEMY engine

In [ ]:
config = dotenv_values("./.env")
username = config.get("DATABASE_USERNAME")
password = config.get("DATABASE_PASSWORD")
dbname = config.get("DATABASE_NAME")
port = config.get("DATABASE_PORT")
host = config.get("DATABASE_HOST")

engine = create_engine(f"postgresql+psycopg2://{username}:{password}@{host}:{port}/{dbname}", echo=True)

## Get path from categories

In [82]:
path_data = os.path.abspath(os.path.join(os.getcwd(), "../oracle_data"))
arquivos_cor_raca = os.path.abspath(os.path.join(path_data, "CorRaa"))
arquivos_depend_admin = os.path.abspath(os.path.join(path_data, "DependnciaAdministrativa"))

print(arquivos_cor_raca)
diretorio, desmembrado = os.path.split(arquivos_cor_raca)
print(desmembrado)
print(os.listdir(arquivos_cor_raca))

/home/jorranico/Documentos/UnB/Disciplinas/2024-1-EducaMinas-backend/WebScrapper/oracle_data/CorRaa
CorRaa
['2022_EnsinoFundamental_AnosFinais_CorRaa.csv', '2020_EnsinoFundamental_TodososValoresdeColunas_CorRaa.csv', '2023_EnsinoFundamental_AnosIniciais_CorRaa.csv', '2021_EnsinoFundamental_AnosFinais_CorRaa.csv', '2023_EnsinoFundamental_TodososValoresdeColunas_CorRaa.csv', '2021_EnsinoFundamental_TodososValoresdeColunas_CorRaa.csv', '2022_EnsinoFundamental_AnosIniciais_CorRaa.csv', '2023_EnsinoMdio__CorRaa.csv', '2021_EnsinoFundamental_AnosIniciais_CorRaa.csv', '2022_EnsinoFundamental_TodososValoresdeColunas_CorRaa.csv', '2020_EnsinoFundamental_AnosFinais_CorRaa.csv', '2021_EnsinoMdio__CorRaa.csv', '2020_EnsinoMdio__CorRaa.csv', '2022_EnsinoMdio__CorRaa.csv', '2023_EnsinoFundamental_AnosFinais_CorRaa.csv', '2020_EnsinoFundamental_AnosIniciais_CorRaa.csv']


## DATA INTERFACE

In [69]:
path = '2020_EnsinoFundamental_TodososValoresdeColunas_DependnciaAdministrativa.csv'
#'2020_EnsinoFundamental_TodososValoresdeColunas_DependnciaAdministrativa.csv'
#'2023_EnsinoMdio__DependnciaAdministrativa.csv'



if not '__' in path:
    padrao = r'(\d{4})_(\w+)_(\w+)_(\w+)\.csv'
else:
    padrao = r'(\d{4})_(\w+)__(\w+)\.csv'
    
teste = re.findall(padrao, path)[0]
print(teste)

#ESTA CELULA É DESCARTAVEL

('2020', 'EnsinoFundamental', 'TodososValoresdeColunas', 'DependnciaAdministrativa')


- transformar tudo para o tipo de dado do banco de dados, se possivel com SQLALCHEMY

In [80]:
class Filters():
    # csv_path is path source folder + file name
    def __init__(self, csv_path) -> None:
        self.__file_path = csv_path
        source_path, self.__file_name = os.path.split(csv_path)
        if not '__' in self.__file_name:
            pattern = r'(\d{4})_(\w+)_(\w+)_(\w+)\.csv'
        else:
            pattern = r'(\d{4})_(\w+)__(\w+)\.csv'
        self.__filters = re.findall(pattern, self.__file_name)[0]
        
        self.__correspondents = {
            'EnsinoFundamental':'EF',
            'AnosIniciais':'1',
            'AnosFinais':'2',
            'TodososValoresdeColunas':'todos',
            'EnsinoMdio':'EM',
            'CorRaa':'cor_raca',
            'DependnciaAdministrativa':'dependencia_administrativa'
        }

        self.__year()
        self.__teaching_stage()
        self.__category()

    def __year(self):
        self.__year = int(self.__filters[0])

    def __teaching_stage(self):
        self.__teaching_stage = self.__correspondents[self.__filters[1]]
        if self.__teaching_stage == 'EF':
            level = self.__correspondents[self.__filters[2]]
            if level != 'todos':
                self.__teaching_stage += level

    def __category(self):
        self.__category = self.__correspondents[self.__filters[-1]]
    
    #getters
    def get_year(self):
        return self.__year
    
    def get_teaching_stage(self):
        return self.__teaching_stage
    
    def get_category(self):
        return self.__category
    
    def get_file_name(self):
        return self.__file_name
    
    def get_file_path(self):
        return self.__file_path

    """ def __municipio_id(self):
        pass """



In [83]:
path = '/home/jorranico/Documentos/UnB/Disciplinas/2024-1-EducaMinas-backend/WebScrapper/oracle_data/CorRaa/2022_EnsinoMdio__CorRaa.csv'
arquivo1 = Filters(path)
print(arquivo1.get_teaching_stage())
print(arquivo1.get_year())
print(arquivo1.get_category())
print(arquivo1.get_file_path())

EM
2022
cor_raca
/home/jorranico/Documentos/UnB/Disciplinas/2024-1-EducaMinas-backend/WebScrapper/oracle_data/CorRaa/2022_EnsinoMdio__CorRaa.csv
